# Imports

In [ ]:
import os, sys
sys.path.append(os.path.abspath('../../two_dim_majoranas/'))

import hpc05
import holoviews as hv

import numpy as np
import kwant
import scipy.constants
import cmath
import functools as ft
from itertools import product

import plotting_results
import adaptive
adaptive.notebook_extension()

import sns_system, spectrum, topology, supercurrent

from IPython.core.magic import register_cell_magic
import ipywidgets as widgets
run_cluster = widgets.Checkbox(
    value=False,
    description='Check to run cluster',
    disabled=False
)

interactive_shell = get_ipython()
@register_cell_magic
def cluster_mark_checked(line, cell):
    if run_cluster.value is True:
        run_cluster.value = False
        interactive_shell.run_cell(cell)
    else:
        return "Command not run. Check the box above to run."

del(cluster_mark_checked)
run_cluster

In [ ]:
%%cluster_mark_checked
# client, dview, lview = hpc05.connect_ipcluster(20, folder='two_dim_majoranas')
# hpc05.kill_remote_ipcluster()
client, dview, lview = hpc05.start_remote_and_connect(100, folder='two_dim_majoranas')
client[:].use_pickle()

In [ ]:
import numpy as np
import scipy.constants
import cmath

import functools as ft
import kwant
import sns_system, spectrum, topology, supercurrent


In [ ]:
constants = dict(
    m_eff=0.02 * scipy.constants.m_e / (scipy.constants.eV * 1e-3) / 1e18,  # effective mass in kg, 
    hbar=scipy.constants.hbar / (scipy.constants.eV * 1e-3),
    e = scipy.constants.e,
    current_unit=scipy.constants.k * scipy.constants.e / scipy.constants.hbar * 1e9,  # to get nA
    mu_B=scipy.constants.physical_constants['Bohr magneton'][0] / (scipy.constants.eV * 1e-3),
    k=scipy.constants.k / (scipy.constants.eV * 1e-3),
    exp=cmath.exp,
    cos=cmath.cos,
    sin=cmath.sin
   )

# Make zigzag shaped system

In [ ]:
# %%px

def l_shaped_system(a, L_m, L_x, z_x, z_y, W_up, W_down, edge_thickness=1,
                    transverse_soi=True,
                    mu_from_bottom_of_spin_orbit_bands=True,
                    k_x_in_sc=False, leaded=True, **_):

    #     HAMILTONIAN DEFINITIONS
    template_strings = sns_system.get_template_strings(
        transverse_soi, mu_from_bottom_of_spin_orbit_bands, k_x_in_sc)
    
    conservation_matrix = -supercurrent.sigz
    
    # TURN HAMILTONIAN STRINGS INTO TEMPLATES
    kwargs = dict(coords=('x', 'y'), grid_spacing=a)
    template_barrier = kwant.continuum.discretize(
        template_strings['ham_barrier'], **kwargs)
    template_normal = kwant.continuum.discretize(
        template_strings['ham_normal'], **kwargs)
    template_sc_left = kwant.continuum.discretize(
        template_strings['ham_sc_left'], **kwargs)
    template_sc_right = kwant.continuum.discretize(
        template_strings['ham_sc_right'], **kwargs)

    def union_shape(shapes):
        def _shape(pos):
            res = False
            for shape in shapes:
                res |= shape(pos)
            return res
        return _shape
        
    def intersection_shape(shape_A, shape_B):
        def _shape(pos):
            return shape_A(pos) and not shape_B(pos)
        return _shape
    
    def below_zigzag(z_x, z_y, offset):
        def shape(site):
            x, y = site.pos - offset
            if 0 <= x < 2*z_x:
                return y <  a * ((z_y* np.sin(np.pi*2 * x / z_x))//a)
            else:
                return False
        return shape
    
    def above_zigzag(z_x, z_y, offset):
        def shape(pos):
            return not below_zigzag(z_x, z_y, offset)(pos)
        return shape
    
    def within_zigzag(z_x, z_y, offset):
        def shape(pos):
            x, y = pos.pos
            return below_zigzag(z_x, z_y, (offset[0], offset[1] + L_m))(pos) and above_zigzag(z_x, z_y, offset)(pos) and 0<=x<L_x
        return shape
    
    def edge_zigzag(z_x, z_y, offset):
        def shape(pos):
            x, y = pos.pos
            return ((below_zigzag(z_x, z_y, (offset[0], offset[1] + L_m))(pos) and above_zigzag(z_x, z_y, (offset[0], offset[1] + L_m-a*edge_thickness))(pos) and 0<=x<L_x)
                    or
                   (below_zigzag(z_x, z_y, (offset[0], offset[1] +edge_thickness*a))(pos) and above_zigzag(z_x, z_y, (offset[0], offset[1]))(pos) and 0<=x<L_x))
        return shape
    
    number_of_zigzags = int(L_x // (2*z_x))+1
    within_zigzag_shapes = [within_zigzag(z_x, z_y, (2*z_x*i, 0)) for i in range(number_of_zigzags)]
    edge_zigzag_shapes = [edge_zigzag(z_x, z_y, (2*z_x*i, 0)) for i in range(number_of_zigzags)]
    edge_shape =union_shape(edge_zigzag_shapes)
    
    middle_shape_with_edge = union_shape(within_zigzag_shapes)
    middle_shape = intersection_shape(middle_shape_with_edge, edge_shape)
    
    def top_shape_block(site):
        x, y = site.pos
        return 0 <= x < L_x and -z_y <= y < L_m + z_y + W_up
    top_shape_with_some_down = intersection_shape(top_shape_block, middle_shape_with_edge)
    
    def down_shape_block(site):
        x, y = site.pos
        return 0 <= x < L_x and -W_down - z_y <= y < z_y
    down_shape_with_some_top = intersection_shape(down_shape_block, middle_shape_with_edge)
    
    top_shape = top_shape_with_some_down
#     intersection_shape(top_shape_with_some_down, down_shape_with_some_top)
    down_shape = down_shape_with_some_top
#     intersection_shape(down_shape_with_some_top, top_shape)
    
    # BUILD FINITE SYSTEM
    syst = kwant.Builder()
    syst.fill(template_normal, middle_shape, (0, L_m//2))
    
    if edge_thickness ==0:
        pass
    
    elif edge_thickness == 1:
        for x in np.arange(0, L_x, a):
            y_up = ((z_y* np.sin(np.pi*2 * x / z_x)+L_m)//a - 1)*a
            y_down = ((z_y* np.sin(np.pi*2 * x / z_x))//a)*a
#             if (x//z_x)%2 == 1:
#                 y = z_y - y
            syst.fill(template_barrier, edge_shape, (x, y_up))
            syst.fill(template_barrier, edge_shape, (x, y_down))
    
    else:  
        syst.fill(template_barrier, edge_shape, (0, 0))
        syst.fill(template_barrier, edge_shape, (0, L_m-a))
    
    if W_up is not 0:
        syst.fill(template_sc_left, top_shape, (0, L_m))
    syst.fill(template_sc_right, down_shape, (0, -a))
    
    syst = syst.finalized()

    return syst

In [ ]:
params_raw= dict(g_factor_middle = 26,
                 g_factor_left = 0,
                 g_factor_right = 0,
                 mu = 10,
                 alpha_middle = 20,
                 alpha_left = 0,
                 alpha_right = 0,
                 Delta_left = 2,
                 Delta_right = 2,
                 B = 1.0341,
                 phase = 0,
                 T = 0.0,
                 V = 0,
                 k_x = 0)

syst_pars = dict(a=10, L_m=500, L_x=2000, W_up=200, W_down=200, z_x=1000, z_y=125,
                 offset_lead=100, k_x_in_sc=True, leaded=False, edge_thickness=0)


params = dict(**constants,
              **params_raw)

syst=sns_system.make_zigzag_system(**syst_pars)
plotting_results.plot_syst(syst=syst, params=dict(**params));

In [ ]:
from itertools import product
discrete_variations = dict()
discrete_variations['z_x'] = [500, 750, 1000, 1250, 1500, 2000, 3000]
discrete_variations['z_y'] = [0, 50, 100, 250, 450, 500, 550, 750, 1000]
discrete_variations['B'] = np.linspace(0, 1.5, 51)

pars = {}

for z_x, z_y, B in product(*discrete_variations.values()):
    _syst_pars = syst_pars.copy()
    _params = params.copy()
    _syst_pars['z_x'] = z_x
    _syst_pars['L_x'] = (6000//z_x)*z_x
    _syst_pars['z_y'] = z_y
    _params['B'] = B
    pars[(z_x, z_y, B)] = (_syst_pars, _params)

In [ ]:
res = lview.map(spectrum.calc_lowest_state, pars.values())

In [ ]:
res.done()



# res.result()


In [ ]:
len(res.submitted)

In [ ]:
res.progress




In [ ]:
res.result()

In [ ]:
data=dict(zip(pars.keys(), res.result()))

In [ ]:
N_z_x = len(discrete_variations['z_x'])
N_z_y = len(discrete_variations['z_y'])
N_B = len(discrete_variations['B'])
Z_0s = {}
Z_1s = {}

for B in discrete_variations['B']:
    Z_0 = np.zeros((N_z_x, N_z_y))
    Z_1 = np.zeros((N_z_x, N_z_y))
    for idx_x, z_x in enumerate(discrete_variations['z_x']):
        for idx_y, z_y in enumerate(discrete_variations['z_y']):
            key = (z_x, z_y, B)
            Z_0[idx_x, idx_y] = data[key][0]
            Z_1[idx_x, idx_y] = data[key][1]
            if z_x == 1000 and z_y == 125:
                print(B, data[key][0], data[key][1])
    Z_0s[B] = Z_0
    Z_1s[B] = Z_1
    

In [ ]:
plot_dict = {('lowest', k): hv.Image(np.flip(v,1), bounds=[discrete_variations['z_x'][0], discrete_variations['z_y'][0],
                          discrete_variations['z_x'][-1],discrete_variations['z_y'][-1]]) for k,v in Z_0s.items()}
plot_dict.update({('second lowest', k): hv.Image(np.flip(v,1), bounds=[discrete_variations['z_x'][0], discrete_variations['z_y'][0],
                          discrete_variations['z_x'][-1],discrete_variations['z_y'][-1]]) for k,v in Z_0s.items()})

In [ ]:
[1,1,2,2][::2]

In [ ]:
%%opts Image [colorbar=True width=400] (cmap='Viridis')
(hv.HoloMap(plot_dict, kdims=['which eigenmode', 'B']))

In [ ]:
Z_

In [ ]:
%opts?

In [ ]:
print(res.done(), res.started, res.elapsed)


In [ ]:
def bands_wrap(k):
    ham = syst.hamiltonian_submatrix(sparse=True, params=dict(**params, k_x=k))
    e, ev = spectrum.sparse_diag(ham, 100, 0)
    return np.sort(e)
    
learner = adaptive.Learner1D(bands_wrap, [-np.pi, np.pi])
runner = adaptive.Runner(learner)
runner.live_info()

In [ ]:
learner.plot()

In [ ]:
ham = syst.hamiltonian_submatrix(params=dict(**params, k_x=0), sparse=True)
d = kwant.operator.Density(syst)
e, ev = spectrum.sparse_diag(ham,4,0)

In [ ]:
e

In [ ]:
ax=plt.axes()
plotting_results.plot_syst(syst=syst, params=dict(**params, k_x=0),ax=ax)
kwant.plotter.map(syst, d(ev[:,0]), ax=ax)
# plt.savefig('no_ohase_difference', dpi=1000)